管道提示词可以将多个提示组合在一起。当您想要重复使用部分提示时，这会很有用。这可以通过 PipelinePrompt 来完成。

PipelinePrompt 由两个主要部分组成：
- 最终提示：返回的最终提示
- 管道提示：元组列表，由字符串名称和提示模板组成。每个提示模板将被格式化，然后作为具有相同名称的变量传递到未来的提示模板。

In [1]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [2]:
full_template = """
{introduction},
{example},
{start}
"""
full_prompt = PromptTemplate.from_template(full_template)


In [3]:
introduction_template = "你正在扮演{person}的角色"
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [5]:
example_template = """
下面是一个交互示例：
Q: {example_a}
A: {example_b}
"""
example_prompt = PromptTemplate.from_template(example_template)


In [6]:
start_template = """现在正式开始：
Q：{input}
A：
"""
start_prompt = PromptTemplate.from_template(start_template)

In [8]:
# 组装
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

pipeline_prompt.input_variables

['example_b', 'example_a', 'person', 'input']

In [10]:
print(pipeline_prompt.format(person="医生", example_a="你叫什么名字？", example_b="我叫小明", input="你今年几岁？"))


你正在扮演医生的角色,

下面是一个交互示例：
Q: 你叫什么名字？
A: 我叫小明
,
现在正式开始：
Q：你今年几岁？
A：




In [15]:
# 接入大模型
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.7)

output_parser = StrOutputParser()

chain = pipeline_prompt | chat | output_parser

chain.invoke({"person": "李世民", "example_a": "你最喜欢的车", "example_b": "贞观八俊", "input": "你最喜欢的社交媒体？"})

'作为李世民，我会回答：“朕生活在唐朝，社交媒体这一概念并不存在于当时的文化之中。但若以当时的形式而言，朕最重视的‘社交平台’便是朝堂与民间的沟通渠道，确保政令通达和民意上呈。”'